## This notebook is to extract fibril site surface *(i.e., Pocket)*

### 1.0 Libraries

In [ ]:
from fibrilsite import *

### 2.0 I/O

In [ ]:
# pocket id
pocket_id = 'Pol_1a_P1'

# start res info
start_resid = 85
start_atom  = 'HB1'
    
# end res info
end_resid = 97
end_atom  = 'HZ1'

# Guard res info
guard_start_resid = 83
guard_start_atom  = 'C'

guard_end_resid   = 97
guard_end_atom    = 'O'

# groove segmentation
start_groove_chains = ['I', 'G', 'C', 'E', 'A']
end_groove_chains   = ['I', 'G', 'C', 'E', 'A']

start_groove_resids = [85,86,87,90,92,95,97,]
end_groove_resids   = [85,86,87,90,92,95,97,]

# path to pdb file
pdb_file = "./6cu7_ABCDEFGHIJ.pdb"

# path to ply file
ply_file = "./MaSIF_output/pred_surfaces/6cu7_ABCDEFGHIJ.ply"

# struct chains
parser = PDBParser(QUIET=1)
struct = parser.get_structure(os.path.basename(pdb_file).split('_')[0], pdb_file)[0]
chains = [chain.id for chain in struct]

In [ ]:
# make output dir
output = os.path.join(os.path.abspath("."), str(datetime.date.today()) +"_"+ str(os.path.basename(pdb_file).split("_")[0]) +"_"+ pocket_id)
os.makedirs(output, exist_ok=False)

In [ ]:
# Data file output
with open(f"{output}/{datetime.date.today()}_{os.path.basename(pdb_file).replace('.pdb','')}_{pocket_id}_params.txt",'w') as w:
    w.write(f'{datetime.date.today()}' + '\n')
    w.write(f'Pocket id: {pocket_id}' + '\n')
    w.write(f'PDB file: {pdb_file}'  + '\n')
    w.write(f'PLY file: {ply_file}'  + '\n')
    w.write(f'Start resid: {start_resid}, Start atom: {start_atom}' + '\n')
    w.write(f'End resid: {end_resid}, End atom: {end_atom}' + '\n')
    w.write(f'Guard start resid: {guard_start_resid}, Guard start atom: {guard_start_atom}' + '\n')
    w.write(f'Guard end resid: {guard_end_resid}, Guard end atom: {guard_end_atom}' + '\n')
    w.write(f'Start groove chains: {start_groove_chains}, Start groove resids: {start_groove_resids}' + '\n')
    w.write(f'End groove chains: {end_groove_chains}, End groove resids: {end_groove_resids}' + '\n')
print('Gedruckt')

### 3.0 Parse

In [ ]:
# parse pdbs and calculate per atom sasa
df_atom_sasa = pdb_parser_and_per_atom_sasa_calculator(
    out_path=output,
    pdb_file=pdb_file,
    probe_r=1.4,
)
print(df_atom_sasa.shape)
df_atom_sasa.head()

In [ ]:
# parse ply file
df_surf = ply_parser(out_path=output, 
                     ply_file=ply_file,
                    )
print(df_surf.shape)
df_surf.head()

### 4.0 Get the main vectors

In [ ]:
# calculate eignenvectors
side, perpend, major = calculate_eigenvectors(pdb_file=pdb_file, 
                                              anchor_residues=[53,], 
                                              anchor_chains= start_groove_chains
                                             )

fibril_axis = np.array(major)
print(f"fibril elongation axis {major}")

In [ ]:
# visualize the major vector
vector_xyz_writer(out_path=output, 
                  file_name='major_eigenvector', 
                  vector=fibril_axis,  
                  element='C',)

### 5.0 Isolate the pocket surface

In [ ]:
# get side surface points
df_side_surf = isolate_fibril_side(
    out_path=output,
    pdb_file=pdb_file,
    df=df_surf,
    fibril_axis=fibril_axis,
)
print(df_side_surf.shape)
df_side_surf.head()

In [ ]:
# visualize the merged points
pymol_xyz_writer(
    out_path=output,
    file_name="side_surface_points",
    df=df_side_surf,
    coords_column="surf_coords",
    element="C",
)

In [ ]:
# isolate the pocket 
df_pocket_coords = pocket_getter(
    df=df_atom_sasa,
    pocket_name=pocket_id,
    pdb_file=pdb_file,
    out_path=output,
    sym=False,
    chains_asym_1=start_groove_chains,
    chains_asym_1_resids=start_groove_resids,
    chains_asym_2=end_groove_chains,
    chains_asym_2_resids=end_groove_resids,
)
print(df_pocket_coords.shape)
df_pocket_coords.head()

In [ ]:
# write out the pocket coords
with open(f"{output}/pocket_coods.xyz", "w") as w:
    for coords in df_pocket_coords['coords']:
        coords = coords.tolist()
        w.write(f'C {coords[0]} {coords[1]} {coords[2]} \n')

In [ ]:
# map surface to atomic coordinates
df_pocket_surf = surface_atomic_coords_mapper(
    out_path=output,
    df_surf=df_side_surf,
    df_atomic=df_pocket_coords,
    threshold=3.0,
    pdb_file=pdb_file,
)
print(df_pocket_surf.shape)
df_pocket_surf.head()

In [ ]:
# visualize the pocket surface
pymol_xyz_writer(
    out_path=output,
    file_name=f"{pocket_id}_pocket_surface",
    df=df_pocket_surf,
    coords_column="surf_coords",
    element="C",
)

### 6.0 Expand to get all surface points

In [ ]:
# Expand on the isolated surf mapped coords
df_full_pocket = the_groove_expander(
    df_pocket_surf=df_pocket_surf,
    df_atom_sasa=df_atom_sasa,
    df_side_surf=df_side_surf,
    pdb_file=pdb_file,
    out_path=output,
    pocket_name=pocket_id,
    threshold=3.0,
)
df_full_pocket.head()

In [ ]:
# visualise the expanded pocket
pymol_xyz_writer(
    out_path=output,
    file_name="extended_pocket_surf",
    df=df_full_pocket,
    coords_column="surf_coords",
    element="C",
)

### 7.0 Clean up the rim

In [ ]:
# get the surf points to remove from the starting end
df_start_rim = pocket_rim_cleaner(df_atom_sasa=df_atom_sasa, df_full_pocket=df_full_pocket, 
                                        pocket_chains=start_groove_chains, res_resid=start_resid, res_atom=start_atom,
                                        guard_resid=guard_start_resid, guard_atom=guard_start_atom, 
                                        threshold=5.0, pdb_file=pdb_file, outpath=output)
df_start_rim.head()

In [ ]:
# get the surf points to remove from the ending end
df_end_rim = pocket_rim_cleaner(df_atom_sasa=df_atom_sasa, df_full_pocket=df_full_pocket, 
                                        pocket_chains=end_groove_chains, res_resid=end_resid, res_atom=end_atom,
                                        guard_resid=guard_end_resid, guard_atom=guard_end_atom, 
                                        threshold=6.0, pdb_file=pdb_file, outpath=output)
df_end_rim.head()

In [ ]:
# Isolate the pocket from the rim points
idx_to_drop = set(df_start_rim.index.tolist() + df_end_rim.index.to_list())

df_pocket_isolate = df_full_pocket.copy()
df_pocket_isolate = df_pocket_isolate[~df_pocket_isolate.index.isin(idx_to_drop)]
df_pocket_isolate.drop_duplicates(subset='MaSIF_index', inplace=True) # remove duplicated points based on MaSIF idx
df_pocket_isolate.reset_index(drop=True, inplace=True)

# Export
df_pocket_isolate.to_csv(f'{output}/{datetime.date.today()}_{os.path.basename(pdb_file).split("_")[0]}_{pocket_id}_isolate.csv')

print(df_pocket_isolate.shape)
df_pocket_isolate.head()

In [ ]:
# visualise the isolated pocket
pymol_xyz_writer(
    out_path=output,
    file_name="isolated_pocket",
    df=df_pocket_isolate,
    coords_column="surf_coords",
    element="C",
)

### 8.0 Manual filtering

In [ ]:
df_pocket_refined = filter_by_range(
    df_atom_sasa=df_atom_sasa,
    df_pocket_isolate=df_pocket_isolate,
    resid=91,
    chain="I",
    atom="CB",
    output=output,
    pocket_id=pocket_id,
    pdb_file=pdb_file,
    dist_thresh=5.0,
)
df_pocket_refined.head()

In [ ]:
df_pocket_refined = filter_by_range(
    df_atom_sasa=df_atom_sasa,
    df_pocket_isolate=df_pocket_refined,
    resid=89,
    chain="I",
    atom="O",
    output=output,
    pocket_id=pocket_id,
    pdb_file=pdb_file,
    dist_thresh=5.0,
)
df_pocket_refined.head()

In [ ]:
pymol_xyz_writer(
    out_path=output,
    file_name="refined_pocket",
    df=df_pocket_refined,
    coords_column="surf_coords",
    element="C",
)

### 9.0 Calculate the area and volume

In [ ]:
# Calculate area from calculated SASA
total_sasa, pocket_hphob_sasa, percent_hphob = get_areas(df=df_pocket_coords)

In [ ]:
# get the surf x,y,z

# map df to the latest stage of pocket form
try:
    df_pocket_refined
except NameError:
    df = df_pocket_isolate.copy()
else:
    df = df_pocket_refined.copy()


df['vertex_x'] = df['surf_coords'].apply(lambda x: float(x[0]))
df['vertex_y'] = df['surf_coords'].apply(lambda x: float(x[1]))
df['vertex_z'] = df['surf_coords'].apply(lambda x: float(x[2]))

# get the coords
mesh_verts = np.stack((df.vertex_x.to_numpy(),
                      df.vertex_y.to_numpy(),
                       df.vertex_z.to_numpy()), axis=1)
mesh_verts.shape

#### 9.1 Create convex hull

In [ ]:
# Create the convexhull
pocket_hull = ConvexHull(points=mesh_verts,)

print(f'pocket area: {pocket_hull.area}')
print(f'pocket volume: {pocket_hull.volume}')

In [ ]:
# visualise
fig = plt.figure(figsize=(10,15), clear=True)
ax = plt.axes(projection='3d')

ax.view_init(-140, 60)
ax.plot3D(mesh_verts[:,0], mesh_verts[:,1],mesh_verts[:,2], '.')

for simplex in pocket_hull.simplices:
    ax.plot3D(mesh_verts[simplex, 0], mesh_verts[simplex, 1], mesh_verts[simplex,2], 'k-')

plt.savefig(f"{output}/{datetime.date.today()}_{pocket_id}_pocket_and_hull.png", format="png" ,dpi=300, transparent=True)

#### 9.2 Export hull with PyMesh

In [ ]:
# create mesh
mesh = pymesh.form_mesh(pocket_hull.points, pocket_hull.simplices) 

In [ ]:
# get the surf nx, ny, nz
df['vertex_nx'] = df['surf_normals'].apply(lambda x: float(x[0]))
df['vertex_ny'] = df['surf_normals'].apply(lambda x: float(x[1]))
df['vertex_nz'] = df['surf_normals'].apply(lambda x: float(x[2]))

# format the other parameters
df['vertex_charge'] = df['surf_charge'].apply(lambda x: float(x))
df['vertex_hbond']  = df['surf_hbond'].apply(lambda x: float(x))
df['vertex_hphob']  = df['surf_hphob'].apply(lambda x: float(x))

# export
df.to_csv(f'{output}/{datetime.date.today()}_pocket_expanded_for_mesh.csv')

print(df.shape)
df.head()

In [ ]:
# get the mesh attr as np arrays
vertex_nx       = df.vertex_nx.to_numpy()
vertex_ny       = df.vertex_ny.to_numpy()
vertex_nz       = df.vertex_nz.to_numpy()
vertex_charge   = df.vertex_charge.to_numpy()
vertex_hbond    = df.vertex_hbond.to_numpy()
vertex_hphob    = df.vertex_hphob.to_numpy()

# add stuff to the mesh
mesh.add_attribute('vertex_nx')
mesh.set_attribute('vertex_nx' , vertex_nx)

mesh.add_attribute('vertex_ny')
mesh.set_attribute('vertex_ny' , vertex_ny)

mesh.add_attribute('vertex_nz')
mesh.set_attribute('vertex_nz' , vertex_nz)

mesh.add_attribute('vertex_charge')
mesh.set_attribute('vertex_charge', vertex_charge)

mesh.add_attribute('vertex_hbond')
mesh.set_attribute('vertex_hbond', vertex_hbond)

mesh.add_attribute('vertex_hphob')
mesh.set_attribute('vertex_hphob', vertex_hphob)

# save mesh with attributes
pymesh.save_mesh(f'{output}/{pocket_id}_convex_hull.ply', mesh, *mesh.get_attribute_names() ,use_float=True, ascii=True)

### 10.0 Write out the pocket properties

In [ ]:
# get the res and id pairs
pocket_resids = sorted(list(set(start_groove_resids+end_groove_resids)))
pocket_resnames  = list(df_atom_sasa[(df_atom_sasa.resid.isin(pocket_resids)) & (df_atom_sasa.atom_type == 'CA') & (df_atom_sasa.chain == start_groove_chains[0])]['resname'])
pocket_res_id_paired = ', '.join([f'{i} {r}' for i,r in zip(pocket_resids, pocket_resnames)])
pocket_res_id_paired

In [ ]:
# content container
prop_dic = {} 

prop_dic['fibril']                  = os.path.basename(pdb_file).split('_')[0]
prop_dic['pocket_id']               = pocket_id 
prop_dic['start_resid']             = start_resid
prop_dic['start_atom']              = start_atom
prop_dic['end_resid']               = end_resid
prop_dic['end_atom']                = end_atom
prop_dic['guard_start_resid']       = guard_start_resid
prop_dic['guard_start_atom']        = guard_start_atom
prop_dic['guard_end_resid']         = guard_end_resid
prop_dic['guard_end_atom']          = guard_end_atom
prop_dic['pocket_chains']           = [list(set(start_groove_chains+end_groove_chains))]
prop_dic['start_groove_chains']     = [start_groove_chains]
prop_dic['end_groove_chains']       = [end_groove_chains]
prop_dic['pocket_seq']              = pocket_res_id_paired
prop_dic['pocket_resids']           = [sorted(list(set(start_groove_resids+end_groove_resids)))]
prop_dic['start_groove_resids']     = [start_groove_resids]
prop_dic['end_groove_resids']       = [end_groove_resids]
prop_dic['pocket_res_num']          = len(set(start_groove_resids + end_groove_resids))
prop_dic['pocket_chain_num']        = len(set(start_groove_chains + end_groove_chains))
prop_dic['pocket_volume(A3)']       = round(pocket_hull.volume, 2)
prop_dic['pocket_area(A2)']         = total_sasa
prop_dic['pocket_hphob_area(A2)']   = pocket_hphob_sasa
prop_dic['pocket_hphob_area_%']     = percent_hphob

prop_dic

#get all into a df
prop_df = pd.DataFrame(prop_dic, index=[0])

# export
prop_df.to_csv(f'{output}/{datetime.date.today()}_{pocket_id}_params.csv')
prop_df